# 读写文件
>有时我们希望保存训练的模型,以备在将来的各种环境中使用,此外当运行一个耗时较长的训练过程时,最佳的做法是定期保存中间结果,以确保在服务器电源被不小心断掉时,我们不会损失几天的计算结果

In [1]:
#加载和保存张量
import torch
from torch import nn
from torch.nn import functional as F

x=torch.arange(4)
torch.save(x,'x-file')
x2=torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [2]:
#我们可以存储一个张量列表,然后把他们读回内存
y=torch.zeros(4)
torch.save([x,y],'x-files')
x2,y2=torch.load('x-files')
(x2,y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [4]:
#写入或读取从字符串映射到张量的子带你
mydict={'x':x,'y':y}
torch.save(mydict,'mydict')
mydict2=torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

# 加载和保存模型参数
>保存单个权重向量确实有用,但是我们要保存整个模型,并在以后加载它们,单独保存每个向量就会变得很麻烦
>往往数百个参数散布在各处,因此深度学习框架提供了内置函数来保存和加载整个网络
>需要注意的一个重要细节是,这将保存模型的参数而不是整个模型

In [6]:
#保存多层感知机的参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.output=nn.Linear(256,10)
    def forward(self,x):
        return self.output(F.relu(self.hidden(x)))

net=MLP()
X=torch.randn(size=(2,20))
Y=net(X)


#我们将模型的参数存储在一个'mlp.params'的文件中
torch.save(net.state_dict(),'mlp.params')

In [10]:
#恢复这个多层感知机
clone=MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()



MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [9]:
#由于两个实例具有相同的模型参数,在输入相同的X时,两个实例的计算结果应该相同
Y_clone=clone(X)
Y_clone==Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])